In [3]:
%load_ext autoreload
%autoreload 2

In [38]:
import os
import sys
import pandas as pd

import statsmodels.api as sm
from statsmodels.formula.api import ols

In [4]:
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [23]:
from src.data_cleaning import cleaning_functions as cfs

In [26]:
from src.data_cleaning import create_dfs as cdfs

# First Shitty Model

In this notebook, I utilised a pre-made function which merges the three tables and isolates down to the logically relevant features.  I will then generate a correlation matrix to inspect which values appear to be most highly correlated with sales price.  I will then create a first shitty model.

In [50]:
df = cdfs.merge_tables()

/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


KeyboardInterrupt: 

In [51]:
df.head()

,saleprice,sqfttotliving,yrbuilt,yrrenovated,bedrooms,zipcode,sqftopenporch,sqftenclosedporch,sqftdeck,heatsystem,...,lakewashington,lakesammamish,smalllakerivercreek,otherview,wfntlocation,trafficnoise,airportnoise,powerlines,othernuisances,adjacentgreenbelt
0,245000,1490,1989,0,3,98002,0,0,0,5,...,0,0,0,0,0,0,0,N,N,N
1,109300,1490,1989,0,3,98002,0,0,0,5,...,0,0,0,0,0,0,0,N,N,N
2,14684,1490,1989,0,3,98002,0,0,0,5,...,0,0,0,0,0,0,0,N,N,N
3,0,1490,1989,0,3,98002,0,0,0,5,...,0,0,0,0,0,0,0,N,N,N
4,0,960,1969,0,3,98092,0,0,0,4,...,0,0,0,0,0,0,0,N,N,N


# Inspect Correlations

Now that we have our data, let's find features that might be correlated with `saleprice`

In [35]:
# create df of correlated values
data_corrs = df.corr()

# inspect all the correlations with 'saleprice' in descedning order so we can see highest correlations
sorted_corrs = data_corrs['saleprice'].sort_values(ascending = False)
sorted_corrs

saleprice              1.000000
sqfttotliving          0.172512
yrbuilt                0.142634
bathfullcount          0.132871
bathhalfcount          0.102335
sqftopenporch          0.094487
bedrooms               0.068942
lakewashington         0.060069
heatsystem             0.055822
territorial            0.052026
seattleskyline         0.033873
olympics               0.027419
wfntlocation           0.027323
otherview              0.021933
lakesammamish          0.021425
bath3qtrcount          0.020641
inadequateparking      0.019853
cascades               0.019628
mtrainier              0.012049
sqftdeck               0.010144
pugetsound             0.005550
airportnoise           0.000642
yrrenovated           -0.002249
heatsource            -0.002669
smalllakerivercreek   -0.005789
sqftenclosedporch     -0.006939
hbuasifvacant         -0.014063
trafficnoise          -0.030762
condition             -0.046319
Name: saleprice, dtype: float64

### Feature selections:

While none are very highly correlated, to start with, let's isolate down to the features that have a correlation coefficient greater that 0.1:

In [37]:
high_corrs = sorted_corrs[sorted_corrs > 0.1]
high_corrs

saleprice        1.000000
sqfttotliving    0.172512
yrbuilt          0.142634
bathfullcount    0.132871
bathhalfcount    0.102335
Name: saleprice, dtype: float64

Create `x`, `y` variables containing independent variables of the high correlated features and dependent variables. 

In [55]:
# create x, y
df_high_corrs = df.loc[:, ['saleprice', 'sqfttotliving', 'yrbuilt', 'bathfullcount', 'bathhalfcount']]

In [72]:
df_high_corrs.head()

,saleprice,sqfttotliving,yrbuilt,bathfullcount,bathhalfcount
0,245000,1490,1989,2,0
1,109300,1490,1989,2,0
2,14684,1490,1989,2,0
3,0,1490,1989,2,0
4,0,960,1969,1,0


# WHY ARE THERE 0 VALUES IN SALEPRICE?  THIS SHOULD HAVE ALREADY BEEN FILTERED OUT!!

## Fuck sake... continuing on with this anyway just to get a model going... BUT COME BACK TO THIS

In [71]:
X = df_high_corrs.drop('saleprice', axis = 1)
y = df['saleprice']

Let's now inspect how the independent variables are correlated and return only those that have correlation higher than 0.1:

In [56]:
corr_x = X.corr()
corr_x[corr_x > 0.1]

,sqfttotliving,yrbuilt,bathfullcount,bathhalfcount
sqfttotliving,1.000000,0.335074,0.574129,0.390974
yrbuilt,0.335074,1.000000,0.389081,0.412973
bathfullcount,0.574129,0.389081,1.000000,0.255087
bathhalfcount,0.390974,0.412973,0.255087,1.000000


### Create multiple linear regression model:

In [74]:
# create formula
formula = 'saleprice ~ '
for col in X:
    formula += col + '+'
    
f = formula[:-1]
f

'saleprice ~ sqfttotliving+yrbuilt+bathfullcount+bathhalfcount'

In [75]:
# create model
model = ols(formula = f, data = df_high_corrs).fit()

#### Let's check this bad boy out:

In [76]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                 1.487e+04
Date:                Mon, 15 Jun 2020   Prob (F-statistic):               0.00
Time:                        20:52:28   Log-Likelihood:            -2.2571e+07
No. Observations:             1492777   AIC:                         4.514e+07
Df Residuals:                 1492772   BIC:                         4.514e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -5.176e+06   5.52e+04    -93.719      0.000   -5.28e+06   -5.07e+06
sqfttotliving   112.9763      0.933    121.098      0.000     111.148     114.805
yrbuilt        2638.4467     28.438     92.780      0.000    2582.710    2694.184
bathfullcount  3.163e+04   1306.796     24.205      0.000    2.91e+04    3.42e+04
bathhalfcount  1.942e+04   1556.627     12.476      0.000    1.64e+04    2.25e+04
==============================================================================
Omnibus:                  3545923.762   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     315420110054.170
Skew:                          23.254   Prob(JB):                         0.00
Kurtosis:                    2254.439   Cond. No.                     2.28e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.28e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""